Test new polytope sampler code and sample loading code

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Load objective

In [12]:
import sys
project_path = "/gpfs/gpfs0/scratch/nrw5cq/CS_4501_Fall22_RL_Project/"
sys.path.append(project_path)
sys.path

['/home/nrw5cq/.local/lib/python3.8/site-packages/PolyRound',
 '/gpfs/gpfs0/scratch/nrw5cq/CS_4501_Fall22_RL_Project/snippets',
 '/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python38.zip',
 '/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8',
 '/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/lib-dynload',
 '',
 '/home/nrw5cq/.local/lib/python3.8/site-packages',
 '/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages',
 '/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/IPython/extensions',
 '/sfs/qumulo/qhome/nrw5cq/.ipython',
 '/gpfs/gpfs0/scratch/nrw5cq/CS_4501_Fall22_RL_Project/',
 '/home/nrw5cq/.local/lib/python3.8/site-packages/libsbml',
 '/gpfs/gpfs0/scratch/nrw5cq/CS_4501_Fall22_RL_Project/']

In [13]:
%pwd

'/gpfs/gpfs0/scratch/nrw5cq/CS_4501_Fall22_RL_Project/snippets'

In [14]:
from my_scripts.optimization import vacc
from ConstrainedLaMCTS.LAMCTS.lamcts import MCTS

In [15]:
import configparser
import multiprocess
import torch

In [16]:
from collections import namedtuple

Arguments = namedtuple('Arguments', 
                       'cfg_dir out_dir name dims threads sim_draws n_init_pts iters samples method load agg_size agg_mapping cp treesize')

args = Arguments(
    "/scratch/nrw5cq/CS_4501_Fall22_RL_Project/config/va_dulles_county/",
    "/scratch/nrw5cq/CS_4501_Fall22_RL_Project/output/",
    "poly_test",
    133,
    20,
    200,
    0,
    1,
    100,
    "lamcts",
    None,
    133,
    "/scratch/nrw5cq/CS_4501_Fall22_RL_Project/config/va_dulles_county/agg_mapping.csv",
    0.1,
    100
)

In [17]:
import os
assert os.path.exists(args.cfg_dir)
assert os.path.exists(args.out_dir)

#config_dir = "config/5_by_5/{}"
config_dir = args.cfg_dir + "{}"

sim_param_config = configparser.ConfigParser()
sim_param_config.optionxform = str  # retain case sensitivity

# load simulation files
with open(config_dir.format("params.cfg")) as config_file:
    sim_param_config.read_file(config_file)
    tsir_config_str = dict(sim_param_config['tsir_config'])
    tsir_config = {}
    for k, v in tsir_config_str.items():
        if k == 'iters':
            tsir_config['iters'] = int(tsir_config_str['iters'])
        else:
            tsir_config[k] = float(tsir_config_str[k])
    opt_config = dict(sim_param_config['opt_config'])
    opt_config['constraint_bnd'] = float(opt_config['constraint_bnd'])

print(tsir_config)

print(opt_config)

# load vaccination/population data and distance matrix
vacc_df = pd.read_csv(config_dir.format("pop.csv"))
dist_list = pd.read_csv(config_dir.format("dist.csv"))
dist_mat = dist_list.pivot(index='zipcode1', columns='zipcode2', values='distKM')
dist_mat = dist_mat.fillna(0)

seed = pd.read_csv(config_dir.format(sim_param_config['seed']['seed']),header=None)
seed = np.array(seed).flatten()


torch.set_num_threads(args.threads)

{'iters': 25, 'tau1': 2.21479534, 'tau2': 0.796080931, 'rho': 1.5609646, 'theta': 3.73211699e-10, 'alpha': 0.97, 'beta': 6.3}
{'obj': 'attacksize', 'V_repr': 'ratio', 'constraint_bnd': 0.05, 'constraint_type': 'ineq', 'aggregate': 'True'}


In [18]:
agg_mapping = pd.read_csv(args.agg_mapping)
agg_mapping = np.array(agg_mapping['mapping'])
v = vacc.VaccProblemLAMCTSWrapper(
    opt_config = opt_config, 
    V_0= vacc_df['vacc'], 
    seed = seed,
    sim_config = tsir_config, 
    pop = vacc_df, 
    distances = np.array(dist_mat),
    negate=True, scale=True,
    cores=args.threads, n_sim=args.sim_draws,
    output_dir = args.out_dir,
    name=args.name,
    agg_vector=agg_mapping,
    agg_size=args.agg_size
)


In [19]:
P = np.zeros(args.agg_size)
for zipcode_index, county_index in enumerate(agg_mapping):
    P[county_index] += vacc_df['pop'][zipcode_index]
c = opt_config['constraint_bnd']
# upper bound by the least vaccinated in each county
ub = np.ones(args.agg_size)*np.inf
for zipcode_index, county_index in enumerate(agg_mapping):
    this_zip_vacc = vacc_df.loc[zipcode_index,'vacc']
    if ub[county_index] > this_zip_vacc:
        ub[county_index] = this_zip_vacc
v.ub = ub
v.lb = np.zeros(args.dims)

agent = MCTS(
     lb = np.zeros(args.dims),      # the lower bound of each problem dimensions
     ub = ub,       # the upper bound of each problem dimensions
     dims = args.dims,              # the problem dimensions
     ninits = 0,           # the number of random samples used in initializations 
     A_ineq = np.array([P]),
     b_ineq = np.array([c*np.sum(P)]),
     A_eq = None, b_eq = None,
     func = v,               # function object to be optimized
     Cp = args.cp,              # Cp for MCTS
     leaf_size = args.treesize, # tree leaf size
     kernel_type = 'linear', #SVM configruation
     gamma_type = "auto",    #SVM configruation
     solver_type = 'turbo',
     num_threads = args.threads
     )

gamma_type: auto
[]
==========collect 0 points for initializing MCTS==========
lb: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ub: [0.90348714 0.90034098 0.89295542 0.9132303  0.91433892 0.8990333
 0.8963964  0.90045613 0.89583333 0.90887934 0.90516145 0.92035876
 0.89841617 0.88343254 0.89801255 0.89468384 0.90105634 0.9178955
 0.91027771 0.85894495 0.90228245 0.86903353 0.9107867  0.89495859
 0.90954774 0.90675063 0.91122947 0.88507719 0.90664962 0.91745076
 0.89401775 0.90645679 0.91559829 0.91438703 0.88954635 0.90974114
 0.91566504 0.89777357 0.8788143  0.90551413 0.92485195 0.90877249
 0.91343432 0.90288066 0.90562871 0.9187

In [20]:
agent.load_samples_from_file(
    "/scratch/nrw5cq/CS_4501_Fall22_RL_Project/output/va_dullesvacc_sim_va_dulles_county_2023-02-12T23:18:01.788022_samples.csv",
    "/scratch/nrw5cq/CS_4501_Fall22_RL_Project/output/va_dullesvacc_sim_va_dulles_county_2023-02-12T23:18:01.788022_best_trace.csv"
)

In [21]:
agent.dynamic_treeify()

----------------------------------------------------------------------------------------------------
node0  is good:False  is leaf:False   val:5543.4399       uct:inf            sp/n:1085/1085  parent:----       kids:node1     node2     
node1  is good:True   is leaf:False   val:6239.5098       uct:6244.9860      sp/n:879/879    parent:node0      kids:node3     node4     
node2  is good:False  is leaf:False   val:2573.3069       uct:2584.6170      sp/n:206/206    parent:node0      kids:node5     node6     
node3  is good:True   is leaf:False   val:7639.3418       uct:7649.0250      sp/n:253/253    parent:node1      kids:node7     node8     
node4  is good:False  is leaf:False   val:5673.7642       uct:5679.9200      sp/n:626/626    parent:node1      kids:node9     node10    
node5  is good:True   is leaf:False   val:3456.9380       uct:3466.6800      sp/n:121/121    parent:node2      kids:node11    node12    
node6  is good:False  is leaf:True    val:1315.4301       uct:1327.0530      

In [22]:
agent.print_tree()

----------------------------------------------------------------------------------------------------
node0  is good:False  is leaf:False   val:5543.4399       uct:inf            sp/n:1085/1085  parent:----       kids:node1     node2     
node1  is good:True   is leaf:False   val:6239.5098       uct:6244.9860      sp/n:879/879    parent:node0      kids:node3     node4     
node2  is good:False  is leaf:False   val:2573.3069       uct:2584.6170      sp/n:206/206    parent:node0      kids:node5     node6     
node3  is good:True   is leaf:False   val:7639.3418       uct:7649.0250      sp/n:253/253    parent:node1      kids:node7     node8     
node4  is good:False  is leaf:False   val:5673.7642       uct:5679.9200      sp/n:626/626    parent:node1      kids:node9     node10    
node5  is good:True   is leaf:False   val:3456.9380       uct:3466.6800      sp/n:121/121    parent:node2      kids:node11    node12    
node6  is good:False  is leaf:True    val:1315.4301       uct:1327.0530      

In [23]:
#agent.search(1087,1086)

In [24]:
root, path = agent.select()
path

=> node1 => node3 => node7 => node13 


[{'A_eq': None,
  'b_eq': None,
  'A_ineq': array([[165077., 149876.,  19918.,  48995.,   4900.,   6228.,  15291.,
           83253., 146783.,  93686., 170388.,   9254.,  30762., 123077.,
           43237.,  84761.,  17931.,  45057.,   4213.,  18264.,  10820.,
           91026.,   6165.,  31919.,   5153.,  81823., 110107.,  10027.,
            4692.,  34222.,  28618., 127064.,   3951.,  19846.,  38718.,
           66181.,  15142.,  25911., 500626.,  49041.,  14691., 110479.,
           74010.,  11750., 139900.,  21661.,  54834.,   3801.,  15469.,
           68687.,   3579., 109147., 102520.,  48466.,  74397.,  33146.,
           93219.,   6623.,  77435.,  55097.,  14675.,  46298.,  35068.,
           64541.,  59968.,   2645.,  55267.,  11809.,  75534.,   1473.,
          294210.,  82691.,   9333.,  33861.,  14788.,  26055.,   1934.,
           56398.,  53143.,  26966., 132188.,  36907.,  30221.,  74726.,
          111996., 327839.,  23385.,  11641.,   3835.,  30107.,  46653.,
         

In [25]:
dim = args.dims
num_constrs = len(path)
A_constr = np.zeros((num_constrs,dim))
b_constr = np.zeros(num_constrs)
global_constrs = path[0]
# we will add the inequality constraint using the hopsy function
if global_constrs["A_ineq"] is not None and global_constrs["b_ineq"] is not None:
    A = global_constrs["A_ineq"]
    b = global_constrs["b_ineq"]
else: # no global constraint
    A = np.array([np.ones(dim)])
    b = np.array([np.inf])

# let the global constraint occupy the first spot...
A_constr[0,:] = A[0]
b_constr[0] = b[0]

# extract constraints from svc objects
for i in range(1,len(path)):
    # TODO: want to be able to change based on kernel type?
    # path is a list of tuples (classifier,0)
    this_classifier = path[i][0].classifier.svm
    # matrix with single row, so extract the vector
    coefs = this_classifier.coef_[0]
    # vector with single entry, so extract the scalar
    intercept = this_classifier.intercept_[0]
    # the set of positive examples for the svm is
    # coefs^T x + intercept >= 0
    # so to rewrite this in hopsy form,
    # -coefs^T x <= intercept
    A_constr[i,:] = coefs
    b_constr[i] = -intercept

In [26]:
A_constr@np.ones(args.dims)*-6

array([-4.46633880e+07, -7.27671031e+01, -7.95213463e+01, -1.89684365e+01,
       -2.14355013e+01])

In [27]:
b_constr

array([ 3.72194900e+05, -6.89649967e-01,  7.61038109e-01, -1.59415681e+00,
        2.03205539e-01])

In [28]:
from ConstrainedLaMCTS.LAMCTS.lamcts import hopsy_sampler

In [29]:
# find the chebyshev center? or the analytic center of this polytope?
A_constr_row_norms = np.linalg.norm(A_constr,axis=1,ord=2)

cheb_A = np.concatenate([A_constr,np.array([A_constr_row_norms]).T],axis=1)
np.shape(cheb_A)

cheb_c = np.zeros(np.shape(cheb_A)[1])
np.put(a=cheb_c,ind=-1,v=-1)

In [30]:
list(zip(v.lb,v.ub))+[(0,None)]

[(0.0, 0.9034871433603382),
 (0.0, 0.9003409795412275),
 (0.0, 0.8929554210236654),
 (0.0, 0.9132303000900964),
 (0.0, 0.914338919925512),
 (0.0, 0.8990332975295381),
 (0.0, 0.8963963963963963),
 (0.0, 0.90045613093641),
 (0.0, 0.8958333333333334),
 (0.0, 0.9088793412101682),
 (0.0, 0.9051614543605392),
 (0.0, 0.9203587637778258),
 (0.0, 0.898416166029492),
 (0.0, 0.8834325396825397),
 (0.0, 0.8980125523012552),
 (0.0, 0.8946838356952738),
 (0.0, 0.901056338028169),
 (0.0, 0.9178955033679228),
 (0.0, 0.9102777118442916),
 (0.0, 0.8589449541284404),
 (0.0, 0.9022824536376604),
 (0.0, 0.8690335305719921),
 (0.0, 0.910786699107867),
 (0.0, 0.8949585936364958),
 (0.0, 0.9095477386934674),
 (0.0, 0.906750629722922),
 (0.0, 0.9112294718153572),
 (0.0, 0.8850771869639794),
 (0.0, 0.9066496163682864),
 (0.0, 0.91745076266729),
 (0.0, 0.8940177510657628),
 (0.0, 0.9064567876478844),
 (0.0, 0.9155982905982906),
 (0.0, 0.914387031408308),
 (0.0, 0.8895463510848126),
 (0.0, 0.909741144414169),
 (0

In [34]:
from scipy.optimize import linprog
soln = linprog(c=cheb_c,A_ub=cheb_A,b_ub=b_constr, bounds = list(zip(v.lb,v.ub))+[(0,None)])

In [35]:
A_constr@soln.x[:-1] <= b_constr

array([ True,  True,  True,  True,  True])

In [36]:
soln.x

array([1.05169840e-10, 1.32608914e-10, 1.09674916e-09, 5.36754124e-10,
       5.18608432e-01, 1.26010186e-09, 2.16799661e-09, 2.57199615e-10,
       1.24449849e-10, 2.03937574e-10, 1.24827666e-10, 2.84386338e-09,
       7.76187957e-10, 1.62321647e-10, 5.33662875e-10, 2.51796082e-10,
       1.47575470e-09, 5.79865802e-10, 1.72355695e-09, 8.52152208e-10,
       1.57345512e-09, 2.29756969e-10, 8.94534050e-10, 7.48055663e-10,
       1.22549921e-09, 2.80589572e-10, 1.73636538e-10, 1.60623808e-09,
       1.49054028e-09, 7.82063672e-10, 9.49043169e-10, 1.49064294e-10,
       2.43322193e-09, 1.80159885e-09, 7.03874397e-10, 3.79298824e-10,
       1.18963908e-09, 7.94391789e-10, 6.23238409e-11, 4.80302210e-10,
       6.93245079e-10, 1.48612257e-10, 3.01701258e-10, 1.43588262e-01,
       1.30295842e-10, 7.21051922e-10, 4.92326186e-10, 9.20547223e-01,
       1.51856343e-09, 3.22722563e-10, 2.12300250e-09, 1.96128144e-10,
       1.88602961e-10, 4.95893031e-10, 3.03550607e-10, 7.28627051e-10,
      

In [37]:
path[1][0].classifier.svm.predict(soln.x[:-1].reshape(1,-1))

array([0], dtype=int32)

In [ ]:
# need to find initial point?

rej_rate, samples = hopsy_sampler.propose_rand_samples_hopsy(
    10000, 
    soln.x[:-1], 
    path, 
    v.lb, v.ub, dim, thin=150, threads=20)

In [ ]:
# need to find initial point?

rej_rate, samples = hopsy_sampler.propose_rand_samples_hopsy(
    10000, 
    None, 
    path, 
    v.lb, v.ub, dim, thin=150, threads=20)

In [88]:
np.shape(np.unique(samples,axis=1))

(10000, 133)

In [120]:
len(np.unique([samples[i,0] for i in range(10000)]))

4265

In [101]:
# try finding the analytic center
def analytic_center_obj(x,A,b):
    # compute every index in the logarithmic potential function... then sum it up
    return -np.sum(np.log(b-A@x))

def analytic_center_gradient(x,A,b,dims):
    # to compute the jth component
    return np.array([np.sum(A[:,j]/(b-A@x)) for j in range(dims)])

In [103]:
analytic_center_gradient(np.ones(133),A_constr,b_constr,133)

array([-0.01050279, -0.04600586,  0.14804348, -0.02675691, -0.05010047,
       -0.03111062,  0.37533491, -0.06996211, -0.03379435, -0.02861627,
       -0.03476289,  0.17951777, -0.07726493, -0.12985057, -0.26286098,
       -0.09570282,  0.19752754, -0.12341609, -0.16884685, -0.13878057,
       -0.11244657, -0.05968135, -0.18065565, -0.06352514, -0.17271715,
       -0.03780922, -0.0261021 ,  0.05586638, -0.43534251, -0.19828589,
       -0.00722438, -0.03541132, -0.34716456,  0.25980516, -0.13529956,
       -0.02613269, -0.06421912, -0.10950719,  0.32851913, -0.06221765,
       -0.06234397, -0.11258836, -0.0600139 ,  0.9135083 , -0.04123599,
       -0.09582982, -0.08658891,  0.34940366,  0.05827534, -0.10970912,
       -0.08866404,  0.16642818, -0.04744865, -0.32075638, -0.05538708,
       -0.08558658, -0.0520238 , -0.70517026, -0.0585442 , -0.07877148,
       -0.25613927, -0.09317525,  0.03387557, -0.03695862, -0.01096416,
       -0.07706369, -0.03070729,  0.26751184, -0.03454719,  0.46

In [97]:
np.ones(4)/np.array([1,2,3,4])

array([1.        , 0.5       , 0.33333333, 0.25      ])

In [113]:
from scipy.optimize import minimize
minimize(
    x0 = soln.x[:-1],
    fun = lambda x: analytic_center_obj(x,A_constr,b_constr),
    jac = lambda x: analytic_center_gradient(x,A_constr,b_constr,133)
)

      fun: nan
 hess_inv: array([[ 9.72581044e-01, -2.46289041e-02, -3.47750488e-03, ...,
        -3.79763388e-03, -2.16492206e-02, -9.63738398e-03],
       [-2.46289041e-02,  9.76970255e-01, -2.54625102e-03, ...,
        -3.60785519e-03, -1.93149592e-02, -1.04313969e-02],
       [-3.47750488e-03, -2.54625102e-03,  9.99193249e-01, ...,
        -3.55539782e-04, -2.82668264e-03, -9.31079020e-05],
       ...,
       [-3.79763388e-03, -3.60785519e-03, -3.55539782e-04, ...,
         9.99431823e-01, -2.96873089e-03, -1.71989254e-03],
       [-2.16492206e-02, -1.93149592e-02, -2.82668264e-03, ...,
        -2.96873089e-03,  9.82891146e-01, -7.35329163e-03],
       [-9.63738398e-03, -1.04313969e-02, -9.31079020e-05, ...,
        -1.71989254e-03, -7.35329163e-03,  9.93140313e-01]])
      jac: array([ 3.98985393e+01,  3.59995311e+01,  5.01039806e+00,  1.12270826e+01,
        3.10019479e+00,  2.44236265e+00,  4.59945602e+00,  2.00406394e+01,
        3.51922621e+01,  2.26168459e+01,  4.10745470e+01